# ICL Evaluation for Larger Qwen Models

Purpose: Evaluate larger Qwen models on HumanEval and HumanEval+ using the optimal 5-shot prompt

This notebook evaluates larger Qwen models (e.g., Qwen2.5-Coder-7B-Instruct or Qwen2.5-Coder-15B-Instruct) on:
- **HumanEval**: 164 Python problems
- **HumanEval+**: Extended version with more test cases

**Using the best prompt configuration from MBPP selection: 5-shot**


In [ ]:
%pip install transformers accelerate datasets tqdm sentencepiece human-eval evalplus --upgrade -q


In [ ]:
import os, re, json, math
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from human_eval.data import read_problems
from tqdm.auto import tqdm

# Ensure math.comb is available (Python 3.8+)
if not hasattr(math, 'comb'):
    def comb(n, k):
        if k > n or k < 0:
            return 0
        if k == 0 or k == n:
            return 1
        k = min(k, n - k)
        result = 1
        for i in range(k):
            result = result * (n - i) // (i + 1)
        return result
    math.comb = comb

SEED = 11667
print("✅ Config loaded")
print(f"  SEED = {SEED}")


✅ Config loaded
  SEED = 11667


## Model Configuration

**Change the model name here to evaluate different model sizes:**
- `Qwen/Qwen2.5-Coder-7B-Instruct` (7B parameters)
- `Qwen/Qwen2.5-Coder-15B-Instruct` (15B parameters)
- Or any other Qwen model you want to test


In [ ]:
# ========== CONFIGURATION ==========
# Change this to the model you want to evaluate
MODEL_NAME = "Qwen/Qwen2.5-Coder-7B-Instruct"  # Change to 15B or other sizes as needed

print(f"Loading model: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

model.eval()
print("✅ Model loaded and set to eval()")


Loading model: Qwen/Qwen2.5-Coder-7B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded and set to eval()


## 5-Shot Prompt Configuration (Best from MBPP Selection)

Using the optimal 5-shot prompt configuration that achieved 35.0% accuracy on MBPP.


In [ ]:
# 5-shot pure code examples (best configuration from MBPP selection)
CODE_EXAMPLE_1 = """def factorial(n):
    if n == 0 or n == 1:
        return 1
    return n * factorial(n - 1)

"""

CODE_EXAMPLE_2 = """def is_palindrome(s):
    s = s.lower().replace(" ", "")
    return s == s[::-1]

"""

CODE_EXAMPLE_3 = """def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    seq = [0, 1]
    for i in range(2, n):
        seq.append(seq[-1] + seq[-2])
    return seq

"""

CODE_EXAMPLE_4 = """def find_max(lst):
    if not lst:
        return None
    m = lst[0]
    for x in lst[1:]:
        if x > m:
            m = x
    return m

"""

CODE_EXAMPLE_5 = """def reverse_list(lst):
    i, j = 0, len(lst) - 1
    while i < j:
        lst[i], lst[j] = lst[j], lst[i]
        i += 1
        j -= 1
    return lst

"""

ALL_CODE_EXAMPLES = [
    CODE_EXAMPLE_1,
    CODE_EXAMPLE_2,
    CODE_EXAMPLE_3,
    CODE_EXAMPLE_4,
    CODE_EXAMPLE_5,
]

# Hard rule for prompt
HARD_RULE = (
    "# You are a Python coding assistant.\n"
    "# Only output valid Python code implementing the required function.\n"
    "# Do NOT use markdown or ```.\n"
    "# Do NOT print explanations or comments outside the function body.\n\n"
)

print("✅ 5-shot prompt configuration loaded")


✅ 5-shot prompt configuration loaded


## Prompt Builder and Code Generation Functions

**Note:** All generation parameters match the original notebook:
- `temperature=0.2`
- `top_p=0.95`
- `max_new_tokens=2048`
- `do_sample=True`


In [ ]:
def get_signature(problem):
    """Extract function signature from HumanEval problem."""
    prompt_lines = problem["prompt"].strip().split("\n")
    first_line = prompt_lines[0] if prompt_lines else ""
    if first_line and not first_line.rstrip().endswith(":"):
        first_line = first_line.rstrip() + ":"
    return first_line + "\n"

def build_task_text_from_humaneval(problem):
    """Extract task description from HumanEval problem docstring."""
    prompt_lines = problem["prompt"].strip().split("\n")
    if len(prompt_lines) > 1:
        docstring = "\n".join(prompt_lines[1:])
        docstring = docstring.strip().strip('"""').strip("'''").strip()
        return docstring
    return "Complete the function"

def build_5shot_prompt(problem):
    """Build 5-shot prompt for HumanEval problem."""
    # 5-shot code prefix
    prefix = "".join(ALL_CODE_EXAMPLES)

    # Task description
    task_text = build_task_text_from_humaneval(problem)
    one_line_task = task_text.replace("\n", " ")

    # Function signature
    sig = get_signature(problem)

    # Combine
    prompt = (
        prefix + "\n" +
        HARD_RULE +
        f"# Task: {one_line_task}\n"
        f"# Implement the following function to solve the task.\n\n"
        f"{sig}"
    )
    return prompt

def generate(model, tokenizer, prompt: str, max_new_tokens: int = 2048, num_samples: int = 1):
    """Generate code from prompt."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    if num_samples == 1:
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.2,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id,
            )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    else:
        generated_texts = []
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.2,
                top_p=0.95,
                pad_token_id=tokenizer.eos_token_id,
                num_return_sequences=num_samples,
            )
        for output in outputs:
            generated_texts.append(tokenizer.decode(output, skip_special_tokens=True))
        return generated_texts

def strip_prompt(full_output: str, prompt: str) -> str:
    """Remove prompt from generated output."""
    if prompt in full_output:
        return full_output.split(prompt, 1)[1].strip()
    return full_output.strip()

def extract_code_from_markdown(text: str) -> str:
    """Extract Python code from possibly-markdown output."""
    if not isinstance(text, str):
        return ""
    s = text.strip()

    # 1) Prefer fenced code blocks
    patterns = [
        r"```python\s*\n(.*?)\n```",
        r"```py\s*\n(.*?)\n```",
        r"```\s*\n(.*?)\n```",
        r"```python\s*(.*?)```",
        r"```\s*(.*?)```",
    ]
    for pat in patterns:
        m = re.search(pat, s, flags=re.DOTALL | re.IGNORECASE)
        if m:
            code = m.group(1).strip()
            if len(code) > 0:
                s = code
                break

    # 2) Remove leading markers
    s = re.sub(r"^###\s*(Instruction|Output|Response):\s*", "", s, flags=re.MULTILINE)
    s = re.sub(r"^(Instruction|Output|Response):\s*", "", s, flags=re.MULTILINE)

    # 3) Find def block
    m_def = re.search(r"(def\s+\w+\([^)]*\):[\s\S]*)", s)
    if m_def:
        s = m_def.group(1).strip()

    # 4) Cut off at instruction markers
    s = s.split("```")[0]
    s = s.split("### Instruction:")[0]
    s = s.split("### Output:")[0]
    s = s.split("### Response:")[0]

    return s.strip()

print("✅ Prompt builder and generation functions ready")


✅ Prompt builder and generation functions ready


In [ ]:
humaneval_problems = read_problems()
print(f"✅ Loaded {len(humaneval_problems)} HumanEval problems")

def check_humaneval(code: str, problem: dict) -> bool:
    """Check HumanEval code correctness using exec()."""
    code_clean = extract_code_from_markdown(code)
    if not code_clean or len(code_clean) < 8:
        return False

    prompt_sig = problem["prompt"]
    test_code = problem["test"]

    if prompt_sig in code_clean:
        body = code_clean.replace(prompt_sig, "")
    else:
        body = code_clean

    full = prompt_sig + body + "\n" + test_code
    try:
        exec(full, {})
        return True
    except Exception:
        return False

def eval_humaneval(model, tokenizer, calculate_pass10: bool = True):
    """Evaluate HumanEval using 5-shot prompt."""
    correct_pass1 = 0
    correct_pass10 = 0
    total = len(humaneval_problems)
    n_samples = 10 if calculate_pass10 else 1

    print(f"🚀 Evaluating HumanEval with 5-shot prompt")
    print(f"   Model: {MODEL_NAME}")
    print(f"   Total problems: {total}")
    if calculate_pass10:
        print(f"   Generating {n_samples} samples per problem for pass@10 calculation")

    for i, (task_id, problem) in enumerate(tqdm(humaneval_problems.items(), desc="HumanEval")):
        prompt = build_5shot_prompt(problem)

        if calculate_pass10:
            full_outs = generate(model, tokenizer, prompt, max_new_tokens=2048, num_samples=n_samples)
            passed_samples = 0
            for full_out in full_outs:
                gen_part = strip_prompt(full_out, prompt)
                code = extract_code_from_markdown(gen_part)
                if check_humaneval(code, problem):
                    passed_samples += 1

            # Pass@1: first sample passes
            if check_humaneval(extract_code_from_markdown(strip_prompt(full_outs[0], prompt)), problem):
                correct_pass1 += 1

            # Pass@10: at least one sample passes
            if passed_samples > 0:
                correct_pass10 += 1
        else:
            full_out = generate(model, tokenizer, prompt, max_new_tokens=2048, num_samples=1)
            gen_part = strip_prompt(full_out, prompt)
            code = extract_code_from_markdown(gen_part)
            if check_humaneval(code, problem):
                correct_pass1 += 1

        if (i + 1) % 10 == 0:
            current_acc1 = correct_pass1 / (i + 1)
            if calculate_pass10:
                current_acc10 = correct_pass10 / (i + 1)
                tqdm.write(f"  Progress: {i+1}/{total} — pass@1={current_acc1:.2%}, pass@10={current_acc10:.2%}")
            else:
                tqdm.write(f"  Progress: {i+1}/{total} — pass@1={current_acc1:.2%}")

    acc_pass1 = correct_pass1 / total
    results = {
        "pass@1": acc_pass1,
        "pass@1_correct": correct_pass1,
        "pass@1_total": total,
    }

    if calculate_pass10:
        acc_pass10 = correct_pass10 / total
        results["pass@10"] = acc_pass10
        results["pass@10_correct"] = correct_pass10
        results["pass@10_total"] = total
        print(f"\n✅ HumanEval pass@1: {acc_pass1:.4f} ({correct_pass1}/{total})")
        print(f"✅ HumanEval pass@10: {acc_pass10:.4f} ({correct_pass10}/{total})")
    else:
        print(f"\n✅ HumanEval pass@1: {acc_pass1:.4f} ({correct_pass1}/{total})")

    return results

# Run evaluation
humaneval_results = eval_humaneval(model, tokenizer, calculate_pass10=True)

# Save results
os.makedirs("results", exist_ok=True)
model_name_short = MODEL_NAME.split("/")[-1].replace("-", "_")
output_file = f"results/{model_name_short}_humaneval_5shot.json"
with open(output_file, "w") as f:
    json.dump({
        "model": MODEL_NAME,
        "prompt_config": "5-shot (best from MBPP selection)",
        "pass@1": float(humaneval_results["pass@1"]),
        "pass@1_correct": int(humaneval_results["pass@1_correct"]),
        "pass@1_total": int(humaneval_results["pass@1_total"]),
        "pass@10": float(humaneval_results.get("pass@10", 0)),
        "pass@10_correct": int(humaneval_results.get("pass@10_correct", 0)),
        "pass@10_total": int(humaneval_results.get("pass@10_total", 0)),
    }, f, indent=2)
print(f"💾 Saved HumanEval results to {output_file}")


✅ Loaded 164 HumanEval problems
🚀 Evaluating HumanEval with 5-shot prompt
   Model: Qwen/Qwen2.5-Coder-7B-Instruct
   Total problems: 164
   Generating 10 samples per problem for pass@10 calculation


HumanEval:   0%|          | 0/164 [00:00<?, ?it/s]

  Progress: 10/164 — pass@1=90.00%, pass@10=90.00%
  Progress: 20/164 — pass@1=70.00%, pass@10=70.00%
  Progress: 30/164 — pass@1=70.00%, pass@10=70.00%
  Progress: 40/164 — pass@1=57.50%, pass@10=57.50%
  Progress: 50/164 — pass@1=46.00%, pass@10=46.00%
  Progress: 60/164 — pass@1=40.00%, pass@10=40.00%
  Progress: 70/164 — pass@1=35.71%, pass@10=37.14%
  Progress: 80/164 — pass@1=32.50%, pass@10=33.75%
  Progress: 90/164 — pass@1=28.89%, pass@10=30.00%
  Progress: 100/164 — pass@1=26.00%, pass@10=27.00%
  Progress: 110/164 — pass@1=23.64%, pass@10=25.45%
  Progress: 120/164 — pass@1=22.50%, pass@10=25.83%
  Progress: 130/164 — pass@1=20.77%, pass@10=23.85%
  Progress: 140/164 — pass@1=19.29%, pass@10=22.14%
  Progress: 150/164 — pass@1=18.67%, pass@10=21.33%
  Progress: 160/164 — pass@1=17.50%, pass@10=20.00%

✅ HumanEval pass@1: 0.1707 (28/164)
✅ HumanEval pass@10: 0.1951 (32/164)
💾 Saved HumanEval results to results/Qwen2.5_Coder_7B_Instruct_humaneval_5shot.json


In [ ]:
humaneval_problems = read_problems()
print(f"✅ Loaded {len(humaneval_problems)} HumanEval problems")   # 在notebook中添加调试代码
for i in range(5):
    problem = list(humaneval_problems.items())[i]
    prompt = build_5shot_prompt(problem[1])
    output = generate(model, tokenizer, prompt)
    code = extract_code_from_markdown(strip_prompt(output, prompt))
    print(f"Problem {i}:")
    print(f"Generated code:\n{code[:500]}\n")

✅ Loaded 164 HumanEval problems
Problem 0:
Generated code:
def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False

Problem 1:
Generated code:
def separate_paren_groups(paren_string: str) -> List[str]:
    paren_string = paren_string.replace(' ', '')
    result = []
    current_group = []
    depth = 0

    for char in paren_string:
        current_group.append(char)
        if char == '(':
            depth += 1
        elif char == ')':
            depth -= 1
            if depth == 0:
                result.append(''.join(current_group))
                current_group = []

    return result

Problem 2:
Generated code:
return number - int(number)

Problem 3:
Generated code:
def below_zero(operations: List[int]) -> bool:
    balance = 0
    for operation in operations:
        balance += ope

In [ ]:
pip install evalplus

In [ ]:
# Install evalplus if not available, then load HumanEval+
try:
    from evalplus.data import get_human_eval_plus
    humaneval_plus_problems = get_human_eval_plus()
    print(f"✅ Loaded {len(humaneval_plus_problems)} HumanEval+ problems")
except ImportError:
    print("⚠️  evalplus not installed. Installing now...")
    import subprocess
    import sys
    import importlib
    subprocess.check_call([sys.executable, "-m", "pip", "install", "evalplus", "-q"])
    print("✅ evalplus installed, reloading...")
    # Reload the module if it was partially loaded
    if 'evalplus' in sys.modules:
        del sys.modules['evalplus']
    try:
        # Re-attempt import from evalplus.data after install
        from evalplus.data import get_human_eval_plus
    except ImportError:
        # Fallback to direct submodule import if top-level still fails
        from evalplus.data.humaneval import get_human_eval_plus
    humaneval_plus_problems = get_human_eval_plus()
    print(f"✅ Loaded {len(humaneval_plus_problems)} HumanEval+ problems")
except Exception as e:
    print(f"⚠️  Failed to load HumanEval+: {e}")
    print("   You can skip HumanEval+ evaluation if needed")
    humaneval_plus_problems = None

def check_humaneval_plus(code: str, problem: dict) -> bool:
    """Check HumanEval+ code correctness using exec()."""
    code_clean = extract_code_from_markdown(code)
    if not code_clean or len(code_clean) < 8:
        return False

    prompt_sig = problem["prompt"]
    test_code = problem["test"]

    if prompt_sig in code_clean:
        body = code_clean.replace(prompt_sig, "")
    else:
        body = code_clean

    full = prompt_sig + body + "\n" + test_code
    try:
        exec(full, {})
        return True
    except Exception:
        return False

def eval_humaneval_plus(model, tokenizer, calculate_pass10: bool = True):
    """Evaluate HumanEval+ using 5-shot prompt."""
    if humaneval_plus_problems is None:
        print("❌ HumanEval+ not loaded. Skipping.")
        return None

    correct_pass1 = 0
    correct_pass10 = 0
    total = len(humaneval_plus_problems)
    n_samples = 10 if calculate_pass10 else 1

    print(f"🚀 Evaluating HumanEval+ with 5-shot prompt")
    print(f"   Model: {MODEL_NAME}")
    print(f"   Total problems: {total}")
    if calculate_pass10:
        print(f"   Generating {n_samples} samples per problem for pass@10 calculation")

    for i, (task_id, problem) in enumerate(tqdm(humaneval_plus_problems.items(), desc="HumanEval+")):
        prompt = build_5shot_prompt(problem)

        if calculate_pass10:
            full_outs = generate(model, tokenizer, prompt, max_new_tokens=2048, num_samples=n_samples)
            passed_samples = 0
            for full_out in full_outs:
                gen_part = strip_prompt(full_out, prompt)
                code = extract_code_from_markdown(gen_part)
                if check_humaneval_plus(code, problem):
                    passed_samples += 1

            # Pass@1: first sample passes
            if check_humaneval_plus(extract_code_from_markdown(strip_prompt(full_outs[0], prompt)), problem):
                correct_pass1 += 1

            # Pass@10: at least one sample passes
            if passed_samples > 0:
                correct_pass10 += 1
        else:
            full_out = generate(model, tokenizer, prompt, max_new_tokens=2048, num_samples=1)
            gen_part = strip_prompt(full_out, prompt)
            code = extract_code_from_markdown(gen_part)
            if check_humaneval_plus(code, problem):
                correct_pass1 += 1

        if (i + 1) % 10 == 0:
            current_acc1 = correct_pass1 / (i + 1)
            if calculate_pass10:
                current_acc10 = correct_pass10 / (i + 1)
                tqdm.write(f"  Progress: {i+1}/{total} — pass@1={current_acc1:.2%}, pass@10={current_acc10:.2%}")
            else:
                tqdm.write(f"  Progress: {i+1}/{total} — pass@1={current_acc1:.2%}")

    acc_pass1 = correct_pass1 / total
    results = {
        "pass@1": acc_pass1,
        "pass@1_correct": correct_pass1,
        "pass@1_total": total,
    }

    if calculate_pass10:
        acc_pass10 = correct_pass10 / total
        results["pass@10"] = acc_pass10
        results["pass@10_correct"] = correct_pass10
        results["pass@10_total"] = total
        print(f"\n✅ HumanEval+ pass@1: {acc_pass1:.4f} ({correct_pass1}/{total})")
        print(f"✅ HumanEval+ pass@10: {acc_pass10:.4f} ({correct_pass10}/{total})")
    else:
        print(f"\n✅ HumanEval+ pass@1: {acc_pass1:.4f} ({correct_pass1}/{total})")

    return results

# Run evaluation
humaneval_plus_results = eval_humaneval_plus(model, tokenizer, calculate_pass10=True)

# Save results
if humaneval_plus_results is not None:
    os.makedirs("results", exist_ok=True) # Ensure the directory exists
    model_name_short = MODEL_NAME.split("/")[-1].replace("-", "_")
    output_file = f"results/{model_name_short}_humaneval_plus_5shot.json"
    with open(output_file, "w") as f:
        json.dump({
            "model": MODEL_NAME,
            "prompt_config": "5-shot (best from MBPP selection)",
            "pass@1": float(humaneval_plus_results["pass@1"]),
            "pass@1_correct": int(humaneval_plus_results["pass@1_correct"]),
            "pass@1_total": int(humaneval_plus_results["pass@1_total"]),
            "pass@10": float(humaneval_plus_results.get("pass@10", 0)),
            "pass@10_correct": int(humaneval_plus_results.get("pass@10_correct", 0)),
            "pass@10_total": int(humaneval_plus_results.get("pass@10_total", 0))
        }, f, indent=2)
    print(f"💾 Saved HumanEval+ results to {output_file}")

In [ ]:
# Print summary
print("="*70)
print("📊 Evaluation Results Summary")
print("="*70)
print(f"Model: {MODEL_NAME}")
print(f"Prompt: 5-shot (best from MBPP selection)")
print()
print("HumanEval:")
print(f"  Pass@1:  {humaneval_results['pass@1']:.4f} ({humaneval_results['pass@1_correct']}/{humaneval_results['pass@1_total']})")
print(f"  Pass@10: {humaneval_results.get('pass@10', 0):.4f} ({humaneval_results.get('pass@10_correct', 0)}/{humaneval_results.get('pass@10_total', 0)})")
print()
if humaneval_plus_results is not None:
    print("HumanEval+:")
    print(f"  Pass@1:  {humaneval_plus_results['pass@1']:.4f} ({humaneval_plus_results['pass@1_correct']}/{humaneval_plus_results['pass@1_total']})")
    print(f"  Pass@10: {humaneval_plus_results.get('pass@10', 0):.4f} ({humaneval_plus_results.get('pass@10_correct', 0)}/{humaneval_plus_results.get('pass@10_total', 0)})")
print("="*70)


📊 Evaluation Results Summary
Model: Qwen/Qwen2.5-Coder-7B-Instruct
Prompt: 5-shot (best from MBPP selection)

HumanEval:
  Pass@1:  0.1707 (28/164)
  Pass@10: 0.1951 (32/164)

